<a href="https://colab.research.google.com/github/Prabhu-04/ML-Algorithms-from-Scratch/blob/master/MultiVariableLinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import math, random
from functools import partial

In [43]:
def mean(x):
  return sum(x)/len(x)
def dot(x,y):
  return sum(x_i*y_i for x_i,y_i in zip(x,y))
def sum_of_squares(x):
  return dot(x,x)
def de_mean(x):
  x_bar=mean(x)
  return [x_i-x_bar for x_i in x]
def variance(x):
  deviations=de_mean(x)
  n=len(x)
  return sum_of_squares(deviations)/(n-1)
def standard_deviation(x):
  return math.sqrt(variance(x))
def covariance(x,y):
  n=len(x)
  return dot(de_mean(x),de_mean(y))/(n-1)
def correlation(x,y):
  x_stdev=standard_deviation(x)
  y_stdev=standard_deviation(y)
  if x_stdev > 0 and y_stdev > 0:
    return covariance(x,y)/x_stdev/y_stdev
  else :return 0
def vector_subtract(v, w):
    return [v_i - w_i for v_i, w_i in zip(v,w)]
def vector_add(x,y):
  return [x_i+y_i for x_i,y_i in zip(x,y)]
def scalar_multiply(c, v):
    return [c * v_i for v_i in v]
def in_random_order(data):
    """generator that returns the elements of data in random order"""
    indexes = [i for i, _ in enumerate(data)]  # create a list of indexes
    random.shuffle(indexes)                    # shuffle them
    for i in indexes:                          # return the data in that order
        yield data[i]
def total_sum_of_squares(y):
  return sum(y_i**2 for y_i in de_mean(y))
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2
def median(v):
    n = len(v)
    sorted_v = sorted(v)
    midpoint = n // 2

In [44]:
def minimize_stochastic(target_fn, gradient_fn, x, y, theta_0, alpha_0=0.01):

    data = list(zip(x, y))
    theta = theta_0                             # initial guess
    alpha = alpha_0                             # initial step size
    min_theta, min_value = None, float("inf")   # the minimum so far
    iterations_with_no_improvement = 0

    # if we ever go 100 iterations with no improvement, stop
    while iterations_with_no_improvement < 100:
        value = sum( target_fn(x_i, y_i, theta) for x_i, y_i in data )

        if value < min_value:
            # if we've found a new minimum, remember it
            # and go back to the original step size
            min_theta, min_value = theta, value
            iterations_with_no_improvement = 0
            alpha = alpha_0
        else:
            # otherwise we're not improving, so try shrinking the step size
            iterations_with_no_improvement += 1
            alpha *= 0.9

        # and take a gradient step for each of the data points
        for x_i, y_i in in_random_order(data):
            gradient_i = gradient_fn(x_i, y_i, theta)
            theta = vector_subtract(theta, scalar_multiply(alpha, gradient_i))

    return min_theta

In [45]:
def predict(x_i,beta):
  return dot(x_i,beta)

In [46]:
def error(x_i,y_i,beta):
  return y_i - predict(x_i,beta)

In [47]:
def squared_error(x_i,y_i,beta):
  return error(x_i,y_i,beta)**2

In [48]:
def squared_error_gradient(x_i,y_i,beta):
  return [-2 * xij * error(x_i,y_i,beta) for xij in x_i]

In [49]:
def estimate_beta(x,y):
  beta_initial= [random.random() for x_i in x[0]]
  return minimize_stochastic(squared_error,squared_error_gradient,x,y,beta_initial,0.001)

In [50]:
def multiple_r_squared(x,y,beta):
  sum_of_squared_error= sum(error(x_i,y_i,beta)**2 for x_i,y_i in zip(x,y))
  return 1.0-sum_of_squared_error/total_sum_of_squares(y)

In [51]:
def bootstrap_sample(data):
  return [random.choice(data) for _ in data]

In [52]:
def bootstrap_statistic(data,stats_fn,num_samples):
  return[stats_fn(bootstrap_sample(data)) for _ in range(num_samples)]

In [53]:
def estimate_sample_beta(sample):
    x_sample, y_sample = list(zip(*sample)) # magic unzipping trick
    return estimate_beta(x_sample, y_sample)

In [54]:
def p_value(beta_hat_j, sigma_hat_j):
    if beta_hat_j > 0:
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)

In [55]:
def ridge_penalty(beta,alpha):
  return alpha * dot(beta[1:],beta[1:])

In [56]:
def squared_error_ridge(x_i,y_i,beat,alpha):
  return error(x_i,y_i,beta)**2 + ridge_penalty(beta,alpha)

In [57]:
def ridge_penalty_gradient(beta, alpha):
    """gradient of just the ridge penalty"""
    return [0] + [2 * alpha * beta_j for beta_j in beta[1:]]

In [58]:
def squared_error_ridge_gradient(x_i,y_i,beta,alpha):
  return vector_add(squared_error_gradient(x_i,y_i,beta),ridge_penalty_gradient(beta,alpha))

In [59]:
def estimate_beta_ridge(x,y,alpha):
  beta_initials=[random.random() for x_i in x[0]]
  return minimize_stochastic(partial(squared_error_ridge,alpha=alpha),partial(squared_error_ridge_gradient,alpha=alpha),x,y,beta_initials,0.001)

In [60]:
def lasso_penalty(beta,alpha):
  return alpha * sum(abs(beta_i) for beta_i in beta[1:])

In [64]:
x = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
daily_minutes_spent = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]
random.seed(0)
beta = estimate_beta(x, daily_minutes_spent) # [30.63, 0.972, -1.868, 0.911]
print("beta", beta)
print("r-squared", multiple_r_squared(x, daily_minutes_spent, beta))
print()
print("digression: the bootstrap")
    # 101 points all very close to 100
close_to_100 = [99.5 + random.random() for _ in range(101)]

    # 101 points, 50 of them near 0, 50 of them near 200
far_from_100 = ([99.5 + random.random()] +
                [random.random() for _ in range(50)] +
                [200 + random.random() for _ in range(50)])

print("bootstrap_statistic(close_to_100, median, 100):")
print(bootstrap_statistic(close_to_100, median, 100))
print("bootstrap_statistic(far_from_100, median, 100):")
print(bootstrap_statistic(far_from_100, median, 100))
print()

random.seed(0) # so that you get the same results as me

bootstrap_betas = bootstrap_statistic(list(zip(x, daily_minutes_spent)),
                                          estimate_sample_beta,
                                          100)

bootstrap_standard_errors = [standard_deviation([beta[i] for beta in bootstrap_betas])for i in range(4)]

print("bootstrap standard errors", bootstrap_standard_errors)
print()
print("p_value(30.63, 1.174)", p_value(30.63, 1.174))
print("p_value(0.972, 0.079)", p_value(0.972, 0.079))
print("p_value(-1.868, 0.131)", p_value(-1.868, 0.131))
print("p_value(0.911, 0.990)", p_value(0.911, 0.990))
print()

print("regularization")

random.seed(0)
for alpha in [0.0, 0.01, 0.1, 1, 10]:
  beta = estimate_beta_ridge(x, daily_minutes_spent, alpha=alpha)
  print("alpha", alpha)
  print("beta", beta)
  print("dot(beta[1:],beta[1:])", dot(beta[1:], beta[1:]))
  print("r-squared", multiple_r_squared(x, daily_minutes_spent, beta))
  print()

beta [30.619881701311712, 0.9702056472470465, -1.8671913880379478, 0.9163711597955347]
r-squared 0.6800074955952597

digression: the bootstrap
bootstrap_statistic(close_to_100, median, 100):
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
bootstrap_statistic(far_from_100, median, 100):
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non